In [0]:
# studio imdb 스크래핑 코드
def scrp_prd_company():
  from bs4 import BeautifulSoup
  import requests
  import pandas as pd
  import re

  # meta의 movie_id만 뽑아서 리스트화하기 (스크래핑하기위해서)
  meta=pd.read_csv('meta_movie_cleaned.csv',engine='python')
  mv_id=meta['movie_id'].to_list()

  movie_id=[]
  year=[]
  country=[]
  types=[]
  company=[]


  for i in range(len(mv_id)):
    url='https://www.imdb.com/title/{}/companycredits?ref_=tt_dt_co'.format(mv_id[i])
    resp=requests.get(url)
    html=resp.text
    soup=BeautifulSoup(html,'html5lib')
    l=soup.select('#company_credits_content > ul:nth-of-type(2) > li')

    ls=''
    ls_1=''
    for j in l:
      ls=(j.text)
      result = re.findall('\(([^)]+)', ls)
      if len(result) > 2:
        if result[1]=='USA':
          result0=re.sub('\(([^)]+)','',ls)
          result0_0=re.sub('\)','',result0)
          result1=re.findall('\w+\+?\.?\!?\/?\s.+',result0_0)
          company.append(result1)
          movie_id.append(mv_id[i])
          year.append(result[0])
          country.append(result[1])
          types.append(result[2])
        
        elif result[1]=='World-wide':
          result0=re.sub('\(([^)]+)','',ls)
          result0_0=re.sub('\)','',result0)
          result1=re.findall('\w+\+?\.?\!?\/?\s.+',result0_0)
          company.append(result1)
          movie_id.append(mv_id[i])
          year.append(result[0])
          country.append(result[1])
          types.append(result[2])

        elif (result[2]=='USA') or (result[2]=='World-wide'):
          result0=re.sub('\(([^)]+)','',ls)
          result0_0=re.sub('\)','',result0)
          result1=re.findall('\w+\+?\.?\!?\/?\s.+',result0_0)
          company.append(result1)
          movie_id.append(mv_id[i])
          year.append(result[1])
          country.append(result[2])
          types.append(result[3])
        else:
          pass

  company_ls=[]
  company_str=''
  for i in company:
      company_str=i[0].strip()
      company_ls.append(company_str)

  company_ls
  scrp_prd_company=pd.DataFrame({'movie_id':movie_id,
                                'company':company_ls,
                                'year':year,
                                'country':country,
                                'types':types})

  scrp_prd_company.to_csv('scrp_prd_company_new.csv')


def preprocess_prd_company_new():
  import pandas as pd
  scrp_prd_company=pd.read_csv('scrp_prd_company_new.csv')
  # 년도가 아닌 값들을 결측치 .으로 대체 
  index=pd.read_csv('index_missing_year.csv')
  index_missing_year=index['index'].to_list()
  for i in index_missing_year:
    scrp_prd_company.loc[i,'year']='.'

  # types가 아닌 다른 값이 들어간 관측치 수정
  scrp_prd_company.loc[2763,'types']='all media'
  scrp_prd_company.loc[4644,'types']='theatrical'
  scrp_prd_company.loc[4744,'types']='theatrical'
  scrp_prd_company.loc[6398,'types']='all media'
  scrp_prd_company.loc[6814,'types']='theatrical'
  scrp_prd_company.loc[6933,'types']='all media'
  scrp_prd_company.loc[8032,'types']='theatrical'
  scrp_prd_company.loc[10257,'types']='all media'
  scrp_prd_company.loc[10323,'types']='all media'
  scrp_prd_company.loc[10851,'types']='theatrical'
  scrp_prd_company.loc[12469,'types']='theatrical'
  scrp_prd_company.loc[12613,'types']='theatrical'
  scrp_prd_company.loc[14568,'types']='all media'
  scrp_prd_company.loc[15422,'types']='DVD'
  scrp_prd_company.loc[15434,'types']='theatrical'
  scrp_prd_company.loc[17096,'types']='all media'
  scrp_prd_company.loc[17975,'types']='theatrical'
  scrp_prd_company.loc[18012,'types']='theatrical'
  scrp_prd_company.loc[18013,'types']='all media'
  scrp_prd_company.loc[20477,'types']='theatrical'
  scrp_prd_company.loc[20652,'types']='theatrical'
  scrp_prd_company.loc[20653,'types']='theatrical'
  scrp_prd_company.loc[22171,'types']='theatrical'


  scrp_prd_company.to_csv('scrp_prd_company_new(revised).csv')

# scrp_prd_company()
# preprocess_prd_company_new()

In [0]:
# studio를 새로 스크래핑한 데이터 정제 및 조건에 맞는 movie_id별 데이터 추출

def preprocess_scrp_prd_company():
  # mv_id별 studio 추출 (우선순위 USA-Worldwide/theatrical-all media)
  import pandas as pd
  scrp_prd_company=pd.read_csv('scrp_prd_company_new(revised).csv')
  # scrp_prd_company=scrp_prd_company.drop('Unnamed: 0',axis=1)
  scrp_prd_company=scrp_prd_company.rename({'company':'studio'},axis='columns')

  # movie_meta_cleaned.csv 데이터 갯수 : 5535

  # USA , theatrical만 뽑은 mv_id, studio : 4736 개
  mv_ls=scrp_prd_company['movie_id'].unique()
  mv_ls=mv_ls.tolist()

  com=[]
  mv_id=[]
  for i in mv_ls:
    for j in range(len(scrp_prd_company)):
      if scrp_prd_company['movie_id'][j]==i:
        if (scrp_prd_company['country'][j]=='USA') & (scrp_prd_company['types'][j]=='theatrical'):
          com.append((scrp_prd_company['studio'][j]))
          mv_id.append(scrp_prd_company['movie_id'][j])
          break
  print('usa-theatrical:%s'%(len(mv_id)))

  # for문으로 돌려 찾은 movie_id와 studio 데이터프레임으로 변환
  test=pd.DataFrame({'movie_id':mv_id,
                    'studio':com})

  # USA, theatrical이 아닌 movie_id 구하기 
  A=set(mv_id) # usa, theatrical인 movie_id
  B=set(mv_ls) # 스크래핑한 movie_id
  C=B-A
  # set을 list로 변경
  C=list(C)

  # USA , all media만 뽑은 mv_id, studio : 190 개
  com_1=[]
  mv_id_1=[]
  for i in C:
    for j in range(len(scrp_prd_company)):
      if scrp_prd_company['movie_id'][j]==i:
        if (scrp_prd_company['country'][j]=='USA') & (scrp_prd_company['types'][j]=='all media'):
          com_1.append((scrp_prd_company['studio'][j]))
          mv_id_1.append(scrp_prd_company['movie_id'][j])
          break
  print('usa-all media:%s'%(len(mv_id_1)))

  # for문으로 돌려 찾은 movie_id와 studio 데이터프레임으로 변환
  test1=pd.DataFrame({'movie_id':mv_id_1,
                    'studio':com_1})
  test1.to_csv('test1.csv')

  C=B-A
  D=set(mv_id_1)
  E=C-D

  # World-wide , theatrical만 뽑은 mv_id, studio : 40 개
  E=list(E)

  com_2=[]
  mv_id_2=[]
  for i in E:
    for j in range(len(scrp_prd_company)):
      if scrp_prd_company['movie_id'][j]==i:
        if (scrp_prd_company['country'][j]=='World-wide') & (scrp_prd_company['types'][j]=='theatrical'):
          com_2.append((scrp_prd_company['studio'][j]))
          mv_id_2.append(scrp_prd_company['movie_id'][j])
          break
  print('world-wide, theatrical:%s'%(len(mv_id_2)))
  # for문으로 돌려 찾은 movie_id와 studio 데이터프레임으로 변환
  test2=pd.DataFrame({'movie_id':mv_id_2,
                    'studio':com_2})

  F=set(mv_id_2)
  E=C-D
  G=E-F

  # World-wid , all media만 뽑은 mv_id, studio : 103 개
  G=list(G)

  com_3=[]
  mv_id_3=[]
  for i in G:
    for j in range(len(scrp_prd_company)):
      if scrp_prd_company['movie_id'][j]==i:
        if (scrp_prd_company['country'][j]=='World-wide') & (scrp_prd_company['types'][j]=='all media'):
          com_3.append((scrp_prd_company['studio'][j]))
          mv_id_3.append(scrp_prd_company['movie_id'][j])
          break
  print('world-wide, all media:%s'%(len(mv_id_3)))
  # for문으로 돌려 찾은 movie_id와 studio 데이터프레임으로 변환
  test3=pd.DataFrame({'movie_id':mv_id_3,
                    'studio':com_3})

  G=E-F
  H=set(mv_id_3)
  T=G-H

  # USA , DVD만 뽑은 mv_id, studio : 100 개
  T=list(T)

  com_4=[]
  mv_id_4=[]
  for i in T:
    for j in range(len(scrp_prd_company)):
      if scrp_prd_company['movie_id'][j]==i:
        if (scrp_prd_company['country'][j]=='USA') & (scrp_prd_company['types'][j]=='DVD'):
          com_4.append((scrp_prd_company['studio'][j]))
          mv_id_4.append(scrp_prd_company['movie_id'][j])
          break
  print('usa-dvd:%s'%(len(mv_id_4)))
  # for문으로 돌려 찾은 movie_id와 studio 데이터프레임으로 변환
  test4=pd.DataFrame({'movie_id':mv_id_4,
                    'studio':com_4})

  # 지금까지 만들었던 데이터프레임 병합하고 csv파일생성 
  test_fnl=pd.concat([test,test1,test2,test3,test4])

  # movie_meta_cleaned.csv파일의 movie_id를 불러오기
  movie_meta_cleaned=pd.read_csv('movie_meta_cleaned.csv')
  mv_id_cleaned=pd.DataFrame({'movie_id':movie_meta_cleaned['movie_id']})

  # meta의 movie_id에 studio를 병합
  prd_company_final=pd.merge(mv_id_cleaned,test_fnl,how='left',on='movie_id')
  prd_company_final.to_csv('prd_company_final_new(revised).csv')

# preprocess_scrp_prd_company()

In [0]:
# 최종 prd_company를 현 메타에 merge하고 축약형으로 변경

def studio_final():
  import pandas as pd
  ##### CSV파일 불러오기 위치환경 설정 #####
  meta_cleaned=pd.read_csv('movie_meta_cleaned_ver3.csv') # 여기에 새로운 버전 메타 대입
  prd_com_fnl=pd.read_csv('prd_company_final_new(revised).csv')
  company_count=pd.read_csv('company_count_new.csv')

  # meta_cleaned에 prd_com_fnl을 movie_id에 따라 studio열을 추가
  merged=pd.merge(meta_cleaned,prd_com_fnl,on='movie_id',how='left')

  # 추가한 새로운 열 studio는 맨 마지막 열에 위치 

  studio=company_count['studio'].to_list()
  abb=company_count['abbreviation'].to_list()

  # studio데이터를 축양형으로 바꾸기 (ps 시간조금걸림..한 3분?)
  for i in range(len(studio)):
    for j in range(len(merged)):
      if merged['studio'][j]==studio[i]:
        try:
          merged.loc[merged['studio']==studio[i],'studio']=abb[i]
        except:
          pass

  # null값 결측치 온점으로 변경 
  merged.loc[merged['studio'].isnull(),'studio']='.'
  merged.to_csv('movie_meta_cleaned_ver4.csv')
studio_final()

In [18]:
b=pd.read_csv('movie_meta_cleaned_ver4.csv')
b['studio'].value_counts().reset_index()

,index,studio
0,WB,502
1,Universal,432
2,Paramount,394
3,20th Century Fox,386
4,Sony Pictures/Columbia,378
...,...,...
296,AsianCrsh,1
297,PDA,1
298,NewY,1
299,GabF,1
